In [1]:
import pandas as pd
import os

from pyarrow import parquet as pq

# Ruta del dataset original
dataset_path = "../data_subsets/final_dataset.parquet"

# Carpeta donde se guardarán los chunks
output_dir = "../data_subsets/final_dataset_chunks"
os.makedirs(output_dir, exist_ok=True)

# Leer metadata del parquet
dataset = pq.ParquetFile(dataset_path)

# Número de filas totales y de row groups
n_row_groups = dataset.num_row_groups
print(f"📊 Número de bloques (row groups): {n_row_groups}")

# Guardar cada row group como un chunk parquet separado
for i in range(n_row_groups):
    table = dataset.read_row_group(i)
    df_chunk = table.to_pandas()
    output_path = f"{output_dir}/chunk_{i}.parquet"
    df_chunk.to_parquet(output_path, index=False)
    print(f"✅ Chunk {i} guardado con {len(df_chunk)} filas")


📊 Número de bloques (row groups): 56
✅ Chunk 0 guardado con 1048576 filas
✅ Chunk 1 guardado con 1048576 filas
✅ Chunk 2 guardado con 1048576 filas
✅ Chunk 3 guardado con 1048576 filas
✅ Chunk 4 guardado con 1048576 filas
✅ Chunk 5 guardado con 1048576 filas
✅ Chunk 6 guardado con 1048576 filas
✅ Chunk 7 guardado con 1048576 filas
✅ Chunk 8 guardado con 1048576 filas
✅ Chunk 9 guardado con 1048576 filas
✅ Chunk 10 guardado con 1048576 filas
✅ Chunk 11 guardado con 1048576 filas
✅ Chunk 12 guardado con 1048576 filas
✅ Chunk 13 guardado con 1048576 filas
✅ Chunk 14 guardado con 1048576 filas
✅ Chunk 15 guardado con 1048576 filas
✅ Chunk 16 guardado con 1048576 filas
✅ Chunk 17 guardado con 1048576 filas
✅ Chunk 18 guardado con 1048576 filas
✅ Chunk 19 guardado con 1048576 filas
✅ Chunk 20 guardado con 1048576 filas
✅ Chunk 21 guardado con 1048576 filas
✅ Chunk 22 guardado con 1048576 filas
✅ Chunk 23 guardado con 1048576 filas
✅ Chunk 24 guardado con 1048576 filas
✅ Chunk 25 guardado con

In [2]:
import glob
import pandas as pd

chunk_paths = sorted(glob.glob("../data_subsets/final_dataset_chunks/chunk_*.parquet"))

# Cargar el primer chunk para prueba
df = pd.read_parquet(chunk_paths[0])
print(f"🔍 Chunk cargado con shape: {df.shape}")
display(df.head())


🔍 Chunk cargado con shape: (1048576, 23)


,legId,segment_num,airline_code,aircraft_type,duration_seconds,distance_miles,departure_hour,departure_weekday,cabin,arr_airport,...,fareBasisCode,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,elapsedDays,days_in_advance,totalTravelDistance
0,fe531c56903f83f1a2d3383441628f0e,0,5,18.0,10260.0,1115.0,7,6,4,13,...,2101,0,0,1,305.12,342.6,7,0,41,1115.0
1,fe531c56903f83f1a2d3383441628f0e,0,5,18.0,10260.0,1115.0,7,6,4,13,...,1088,0,0,1,404.65,449.6,7,0,37,1115.0
2,fe531c56903f83f1a2d3383441628f0e,0,5,18.0,10260.0,1115.0,7,6,4,13,...,1562,0,0,1,227.91,259.6,7,0,6,1115.0
3,b7a7c798768db6400d8db954710fca9c,0,7,17.0,7380.0,725.0,8,1,4,1,...,4239,1,0,1,110.70,133.6,9,0,10,725.0
4,b7a7c798768db6400d8db954710fca9c,0,7,17.0,7380.0,725.0,8,1,4,1,...,1586,0,0,1,174.88,202.6,9,0,6,725.0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Supongamos que ya tienes un chunk
X = df.drop(columns=["totalFare", "legId"])  # Elimina columnas no necesarias
y = df["totalFare"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División entrenamiento/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Supongamos que ya tienes un chunk
X = df.drop(columns=["totalFare", "legId"])  # Elimina columnas no necesarias
y = df["totalFare"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División entrenamiento/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)




import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

print("📦 Detectando chunks...")
chunk_files = sorted(glob.glob("../data_subsets/final_dataset_chunks/chunk_*.parquet"))
print(f"📦 Detectados {len(chunk_files)} chunks")

os.makedirs("../data_subsets/train_test", exist_ok=True)

# Inicializamos el scaler
scaler = StandardScaler()

# 1️⃣ Primera pasada: fitting del scaler
print("🧪 Fitting del StandardScaler...")
for i, file in enumerate(chunk_files):
    df = pd.read_parquet(file)

    # Limpiar NaNs
    df = df.dropna(subset=["totalFare", "legId"] + [col for col in df.columns if col not in ["totalFare", "legId"]])

    X = df.drop(columns=["totalFare", "legId"])
    y = df["totalFare"]

    X = np.nan_to_num(X, nan=0.0)
    y = np.nan_to_num(y, nan=0.0)

    # Fitting parcial
    scaler.partial_fit(X)


# Guardar el scaler
joblib.dump(scaler, "../models/standard_scaler.pkl")

# 2️⃣ Segunda pasada: escalar, dividir y guardar
print("✂️ Dividiendo y guardando chunks escalados...")
for i, file in enumerate(chunk_files):
    df = pd.read_parquet(file)
    X = df.drop(columns=["totalFare", "legId"])
    y = df["totalFare"]

    X_scaled = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )

    np.save(f"../data_subsets/train_test/X_train_{i}.npy", X_train)
    np.save(f"../data_subsets/train_test/X_test_{i}.npy", X_test)
    np.save(f"../data_subsets/train_test/y_train_{i}.npy", y_train)
    np.save(f"../data_subsets/train_test/y_test_{i}.npy", y_test)

    print(f"✅ Chunk {i} procesado y guardado: {X_train.shape[0]} train / {X_test.shape[0]} test")

📦 Detectando chunks...
📦 Detectados 56 chunks
🧪 Fitting del StandardScaler...
✂️ Dividiendo y guardando chunks escalados...


/home/cosmopato/flightprice_env/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


✅ Chunk 0 procesado y guardado: 838860 train / 209716 test


/home/cosmopato/flightprice_env/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


✅ Chunk 1 procesado y guardado: 838860 train / 209716 test


...

✅ Chunk 53 procesado y guardado: 838860 train / 209716 test


/home/cosmopato/flightprice_env/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


✅ Chunk 54 procesado y guardado: 838860 train / 209716 test


/home/cosmopato/flightprice_env/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


✅ Chunk 55 procesado y guardado: 838860 train / 209716 test
